In [ ]:
# default_exp constants

# Constants

> Where all constats are stored

This notebook stores all constants.

In [ ]:
#hide
from nbdev.showdoc import *

## Amino Acids
A set of valid amino acids.

In [ ]:
#export
AAs = set('ACDEFGHIKLMNPQRSTUVWY')

In [ ]:
print(AAs)

{'G', 'D', 'V', 'K', 'A', 'N', 'P', 'E', 'Q', 'T', 'C', 'M', 'H', 'I', 'W', 'R', 'F', 'U', 'Y', 'S', 'L'}


## Mass dict
A numba compatible mass dict dictionary

In [ ]:
#export
from numba import types
from numba.typed import Dict

mass_dict = Dict.empty(key_type=types.unicode_type, value_type=types.float64)

mass_dict["A"] = 71.03711379999999
mass_dict["C"] = 103.0091845
mass_dict["D"] = 115.02694309999998
mass_dict["E"] = 129.04259309999998
mass_dict["F"] = 147.0684139
mass_dict["G"] = 57.02146373
mass_dict["H"] = 137.0589119
mass_dict["I"] = 113.08406399999998
mass_dict["K"] = 128.09496299999998
mass_dict["L"] = 113.08406399999998
mass_dict["M"] = 131.04048459999998
mass_dict["N"] = 114.04292749999999
mass_dict["P"] = 97.05276386
mass_dict["Q"] = 128.05857749999998
mass_dict["R"] = 156.101111
mass_dict["S"] = 87.03202843
mass_dict["T"] = 101.0476785
mass_dict["U"] = 150.9536333957
mass_dict["V"] = 99.06841392
mass_dict["W"] = 186.07931299999998
mass_dict["Y"] = 163.06332859999998
mass_dict["cC"] = 160.03064823
mass_dict["oxM"] = 147.03539923
mass_dict["aN"] = 156.05349219
mass_dict["amN"] = 113.058911907
mass_dict["aK"] = 170.10552768999997
mass_dict["pS"] = 166.99835935
mass_dict["pT"] = 181.01400942
mass_dict["pY"] = 243.02965952
mass_dict["deamN"] = 115.02694309299999
mass_dict["deamNQ"] = 129.042593093
mass_dict["cmC"] = 85.9826354
mass_dict["pgE"] = 111.03202840999998
mass_dict["pgQ"] = 111.03202839999999
mass_dict["tmt0N"] = 338.1954054
mass_dict["tmt0K"] = 352.2474409
mass_dict["tmt0Y"] = 387.2158065
mass_dict["tmt2N"] = 339.1987603
mass_dict["tmt2K"] = 353.2507958
mass_dict["tmt2Y"] = 388.21916139999996
mass_dict["tmt6N"] = 343.20585969999996
mass_dict["tmt6K"] = 357.25789519999995
mass_dict["tmt6Y"] = 392.2262608
mass_dict["itraq4KN"] = 258.14533789999996
mass_dict["itraq4K"] = 272.19737339999995
mass_dict["itraq4Y"] = 307.165739
mass_dict["itraq8KN"] = 418.245127
mass_dict["itraq8K"] = 432.2971625
mass_dict["itraq8Y"] = 467.2655281
mass_dict["eK"] = 394.1788305
mass_dict["arg10R"] = 166.10938057776002
mass_dict["arg6R"] = 162.121241
mass_dict["lys8K"] = 136.10916278887998

mass_dict["Electron"] = 0.00054857990907
mass_dict["Proton"] = 1.00727646687
mass_dict["Hydrogen"] = 1.00782503223
mass_dict["C13"] = 13.003354835
mass_dict["Oxygen"] = 15.994914619
mass_dict["OH"] = 17.002739651229998
mass_dict["H2O"] = 18.01056468346
mass_dict["NH3"] = 17.03052

mass_dict["delta_M"] = 1.00286864
mass_dict["delta_S"] = 0.0109135

In [ ]:
# Test that there is an entry for each AA
for _ in AAs:
    assert _ in mass_dict.keys()
    
print(mass_dict['A'])
print(mass_dict['K'])

71.03711379999999
128.09496299999998


In [ ]:
#hide
def test_get_mass_dict():
    m_dict = mass_dict
    assert m_dict["Electron"] == 0.000548579909070

test_get_mass_dict()

In [ ]:
#export

#generates the mass dictionary from table
def get_mass_dict(modfile="../modifications.tsv", aasfile="../amino_acids.tsv"):
    """
    Creates a mass_dict based on tsv files

    Args:
        modfile: Filename of modifications file
        aasfile: Filename of AAs file

    Returns:
        Returns a numba compatible dictionary with masses

    Raises:
        FileNotFoundError: If files are not found
    """
    import pandas as pd

    #for file in [modfile, aasfile]:
        #if not check_file(file):
        #    raise FileNotFoundError("File {} not found.".format(file))

    mods = pd.read_csv(modfile, delimiter="\t")
    aas = pd.read_csv(aasfile, delimiter="\t")

    mass_dict = Dict.empty(key_type=types.unicode_type, value_type=types.float64)

    for identifier, mass in aas[["Identifier", "Monoisotopic Mass (Da)"]].values:
        mass_dict[identifier] = float(mass)

    for identifier, aar, mass in mods[
        ["Identifier", "Amino Acid Residue", "Monoisotopic Mass Shift (Da)"]
    ].values:
        #print(identifier, aar, mass)

        if ("<" in identifier) or (">" in identifier):
            for aa_identifier, aa_mass in aas[["Identifier", "Monoisotopic Mass (Da)"]].values:
                if '^' in aa_identifier:
                    new_identifier = identifier[:-1] + aa_identifier
                    mass_dict[new_identifier] = float(mass) + mass_dict[aa_identifier]
                elif aar == aa_identifier:
                    new_identifier = identifier[:-2] + aa_identifier
                    mass_dict[new_identifier] = float(mass) + mass_dict[aa_identifier]
                else:
                    pass
        else:
            mass_dict[identifier] = float(mass) + mass_dict[aar]

    # Manually add some other masses
    # TODO: Move to file
    mass_dict[
        "Electron"
    ] = (
        0.000548579909070
    )  # electron mass, half a millimass error if not taken into account
    mass_dict["Proton"] = 1.00727646687  # proton mass
    mass_dict["Hydrogen"] = 1.00782503223  # hydrogen mass
    mass_dict["C13"] = 13.003354835  # C13 mass
    mass_dict["Oxygen"] = 15.994914619  # oxygen mass
    mass_dict["OH"] = mass_dict["Oxygen"] + mass_dict["Hydrogen"]  # OH mass
    mass_dict["H2O"] = mass_dict["Oxygen"] + 2 * mass_dict["Hydrogen"]  # H2O mass

    for element in mass_dict:
        print('mass_dict["{}"] = {}'.format(element, mass_dict[element]))

    return mass_dict

In [ ]:
import pandas as pd
## Read modifications from modifications file
mod_db = pd.read_csv('../modifications.tsv', sep='\t')

mods = {}
mods_terminal = {}
mods_protein = {}

for i in range(len(mod_db)):
    mod = mod_db.iloc[i]
    #print(mod)
    if 'terminus' in mod['Type']:
        if 'peptide' in mod['Type']:
            mods_terminal[mod['Identifier']] = mod['Description']
        elif 'protein' in mod['Type']:
            mods_protein[mod['Identifier']] = mod['Description']
        else:
            print('Not understood')
            print(mod['Type'])
            raise
    else:
        mods[mod['Identifier']] = mod['Description']

## Isotopes

We define a `jitclass` that stores the base mass, the number of isotopes, and their abundances. We create the typed dictionary `isotopes` that stores different default isotopes. 

In [ ]:
#export
import numpy as np
from numba import int32, float32, float64, njit, types
from numba.experimental import jitclass
from numba.typed import Dict

spec = [
    ('m0', float32),
    ('dm', int32),
    ('intensities', float32[:]),
]

@jitclass(spec)
class Isotope:
    def __init__(self, m0, dm, intensities):
        self.m0 = m0
        self.dm = dm
        self.intensities = intensities

isotopes = Dict.empty(key_type=types.unicode_type, value_type=Isotope.class_type.instance_type)

isotopes["C"] = Isotope(12, 3, np.array([0.9893, 0.0107, 0.0], dtype=np.float32))
isotopes["H"] = Isotope(1.007940, 3,  np.array([0.999885, 0.000115, 0.0], dtype=np.float32))
isotopes["O"] = Isotope(15.9949146221, 3,  np.array([0.99757, 0.00038, 0.00205], dtype=np.float32))
isotopes["N"] = Isotope(14.0030740052, 2,  np.array([0.99636, 0.00364], dtype=np.float32))
isotopes["S"] = Isotope(31.97207069, 4,  np.array([0.9499, 0.0075, 0.0425, 0.0001], dtype=np.float32))

isotopes["I"] = Isotope(126.904473, 1,  np.array([1], dtype=np.float32))
isotopes["K"] = Isotope(38.9637069, 3,  np.array([0.932581, 0.000117, 0.067302], dtype=np.float32))

In [ ]:
#hide
def test_isotope():
    assert isotopes["C"].m0 == 12
    assert isotopes["C"].dm == 3
    assert np.allclose(isotopes["C"].intensities[0], 0.9893)
    assert np.allclose(isotopes["C"].intensities[1], 0.0107)
    assert np.allclose(isotopes["C"].intensities[2], 0)
    
test_isotope()

## Averagine Model

In [ ]:
#export
averagine_aa = Dict.empty(key_type=types.unicode_type, value_type=types.float64)

averagine_aa["C"] = 4.9384
averagine_aa["H"] = 7.7583
averagine_aa["N"] = 1.3577
averagine_aa["O"] = 1.4773
averagine_aa["S"] = 0.0417

averagine_avg = 111.1254

## Protease dict

A numba compatible dictionary that stores different regular expressions neeeded for digestion. The dictionary was largely taken from the [Pyteomics](https://pyteomics.readthedocs.io/en/latest/_modules/pyteomics/parser.html) website which in turn derived the rules are from [expasy](https://web.expasy.org/peptide_cutter/peptidecutter_enzymes.html).

In [ ]:
#export
protease_dict = Dict.empty(key_type=types.unicode_type, value_type=types.unicode_type)

protease_dict["arg-c"] = "R"
protease_dict["asp-n"] = "\w(?=D)"
protease_dict["bnps-skatole"] = "W"
protease_dict["caspase 1"] = "(?<=[FWYL]\w[HAT])D(?=[^PEDQKR])"
protease_dict["caspase 2"] = "(?<=DVA)D(?=[^PEDQKR])"
protease_dict["caspase 3"] = "(?<=DMQ)D(?=[^PEDQKR])"
protease_dict["caspase 4"] = "(?<=LEV)D(?=[^PEDQKR])"
protease_dict["caspase 5"] = "(?<=[LW]EH)D"
protease_dict["caspase 6"] = "(?<=VE[HI])D(?=[^PEDQKR])"
protease_dict["caspase 7"] = "(?<=DEV)D(?=[^PEDQKR])"
protease_dict["caspase 8"] = "(?<=[IL]ET)D(?=[^PEDQKR])"
protease_dict["caspase 9"] = "(?<=LEH)D"
protease_dict["caspase 10"] = "(?<=IEA)D"
protease_dict["chymotrypsin high specificity"] = "([FY](?=[^P]))|(W(?=[^MP]))"
protease_dict["chymotrypsin low specificity"] = "([FLY](?=[^P]))|(W(?=[^MP]))|(M(?=[^PY]))|(H(?=[^DMPW]))"
protease_dict["clostripain"] = "R"
protease_dict["cnbr"] = "M"
protease_dict["enterokinase"] = "(?<=[DE]{3})K"
protease_dict["factor xa"] = "(?<=[AFGILTVM][DE]G)R"
protease_dict["formic acid"] = "D"
protease_dict["glutamyl endopeptidase"] = "E"
protease_dict["granzyme b"] = "(?<=IEP)D"
protease_dict["hydroxylamine"] = "N(?=G)"
protease_dict["iodosobenzoic acid"] = "W"
protease_dict["lysc"] = "K"
protease_dict["ntcb"] = "\w(?=C)"
protease_dict["pepsin ph1.3"] = "((?<=[^HKR][^P])[^R](?=[FL][^P]))|((?<=[^HKR][^P])[FL](?=\w[^P]))"
protease_dict["pepsin ph2.0"] = "((?<=[^HKR][^P])[^R](?=[FLWY][^P]))|((?<=[^HKR][^P])[FLWY](?=\w[^P]))"
protease_dict["proline endopeptidase"] = "(?<=[HKR])P(?=[^P])"
protease_dict["proteinase k"] = "[AEFILTVWY]"
protease_dict["staphylococcal peptidase i"] = "(?<=[^E])E"
protease_dict["thermolysin"] = "[^DE](?=[AFILMV])"
protease_dict["thrombin"] = "((?<=G)R(?=G))|((?<=[AFGILTVM][AFGILTVWA]P)R(?=[^DE][^DE]))"
protease_dict["trypsin_full"] = "([KR](?=[^P]))|((?<=W)K(?=P))|((?<=M)R(?=P))"
protease_dict["trypsin_exception"] = "((?<=[CD])K(?=D))|((?<=C)K(?=[HY]))|((?<=C)R(?=K))|((?<=R)R(?=[HR]))"
protease_dict["non-specific"] = "()"
protease_dict["trypsin"] = "([KR](?=[^P]))"

In [ ]:
protease_dict["trypsin"]

'([KR](?=[^P]))'

In [ ]:
#hide
def test_get_protease_dict():
    p_dict = protease_dict
    assert p_dict["trypsin"] == "([KR](?=[^P]))"
    
test_get_protease_dict()

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_settings.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_interface.ipynb.
Converted 12_speed.ipynb.
Converted index.ipynb.
